In [3]:
# Import essential libraries
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.optimizers import Adam
from keras.applications import VGG16
from keras.callbacks import ReduceLROnPlateau

ModuleNotFoundError: No module named 'cv2'

In [ ]:
from google.colab import files
# drive.mount('/content/drive')
# uploaded = files.upload()  # Upload all images manually from your local system
DATASET_PATH = '/content/drive/MyDrive/Student-engagement-dataset'

In [ ]:
# Categories based on your dataset structure
CATEGORIES = {
    "Engaged": ["confused", "engaged", "frustrated"],
    "Not_engaged": ["bored", "drowsy", "looking_away"]
}

IMG_SIZE = 70  # Resize images for uniformity
DATASET_PATH = '/content/drive/MyDrive/Student-engagement-dataset'

def load_data():
    data = []
    labels = []

    for main_category, subcategories in CATEGORIES.items():
        for subcategory in subcategories:
            category_path = os.path.join(DATASET_PATH, main_category, subcategory)
            label = 1 if main_category == "Engaged" else 0

            if not os.path.exists(category_path):
                print(f"Warning: Path does not exist - {category_path}")
                continue

            image_count = 0
            for img_filename in os.listdir(category_path):
                img_path = os.path.join(category_path, img_filename)

                if not img_filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    continue

                try:
                    img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    if img_array is None:
                        print(f"Warning: Could not read image - {img_path}")
                        continue

                    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                    # Convert grayscale to RGB
                    rgb_array = cv2.cvtColor(resized_array, cv2.COLOR_GRAY2RGB)
                    data.append(rgb_array)
                    labels.append(label)
                    image_count += 1
                except Exception as e:
                    print(f"Error processing image {img_path}: {e}")

            print(f"Processed {image_count} images in {category_path}")

    # Convert data to NumPy arrays and normalize pixel values
    data = np.array(data) / 255.0
    labels = np.array(labels)

    return data, labels


# Load data and split it into training and testing sets
data, labels = load_data()
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.25, random_state=42)

# Optionally print some information
print(f"Data shape: {data.shape}")
print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def train_and_evaluate_model(X_train, y_train, X_test, y_test, model_name="engagement_model"):
    # Split training data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Load pre-trained VGG16 model (excluding top layers)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

    # Build the custom model
    model = Sequential()
    model.add(base_model)  # Add pre-trained VGG16 model
    model.add(Flatten())  # Flatten the output from the VGG16 base model
    model.add(Dense(512, activation='relu'))  # Add a dense layer with more units
    model.add(Dropout(0.5))  # Dropout for regularization
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

    # Compile the model with a learning rate scheduler
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    datagen.fit(X_train)

    # Learning rate scheduler to reduce learning rate when validation loss plateaus
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Early stopping with more patience
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model with data augmentation
    model.fit(datagen.flow(X_train, y_train, batch_size=32),
              epochs=50,  # Increased number of epochs
              validation_data=(X_val, y_val),
              callbacks=[early_stopping, lr_scheduler])

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    # Save model to Google Drive
    model_path = f'/content/drive/MyDrive/{model_name}.h5'
    model.save(model_path)
    print(f"Model saved to {model_path}")

    # Generate predictions and confusion matrix
    y_pred = model.predict(X_test).flatten()  # Predictions as probabilities
    y_pred_classes = np.round(y_pred).astype(int)  # Convert to binary class (0 or 1)

    # Print confusion matrix
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred_classes))

    # Generate classification report
    print("Classification Report:")
    print(classification_report(y_test, y_pred_classes, target_names=["Not Engaged", "Engaged"], labels=[0, 1]))

    # Generate ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', label=f"ROC Curve (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], color='red', linestyle='--')  # Diagonal line for random guessing
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower right")
    plt.grid()
    plt.show()

# Call the function with your data
train_and_evaluate_model(X_train, y_train, X_test, y_test)


In [ ]:
IMG_SIZE = 70  # Resize images for uniformity

# Prepare the image (resizing and normalization)
def prepare_image(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0

# Model analysis function
def analyze_images(model, test_folder):
    times = []
    attentionspan = []
    start_index = None

    image_files = sorted([f for f in os.listdir(test_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])

    for idx, img_name in enumerate(image_files):
        img_path = os.path.join(test_folder, img_name)
        try:
            prediction = model.predict(prepare_image(img_path))
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            continue

        if prediction[0][0] >= 0.5:
            if start_index is None:
                start_index = idx
        else:
            if start_index is not None:
                end_index = idx
                times.append((start_index, end_index))
                attentionspan.append(end_index - start_index)
                start_index = None

    average_attention_span = sum(attentionspan) / len(attentionspan) if attentionspan else 0
    print("Attention times (indexes):", times)
    print("Average Attention Span (in frames):", average_attention_span)
    print(f"Total periods of attention: {len(times)}")

# Load the model from the file
def load_model(model_path):
    model = tf.keras.models.load_model(model_path)  # Load the model from the path
    return model

# Example usage
model_path = '/content/drive/MyDrive/engagement_model.h5'  # Path to your saved model
test_images_path = '/content/drive/MyDrive/your_test_images'  # Path to your test images

# Load the trained model
model = load_model(model_path)

# Analyze images with the trained model
analyze_images(model, test_images_path)
